In [2]:
import sys
sys.path.append('../unseen')

from dask.distributed import Client, LocalCluster

import myfuncs
import indices

In [3]:
cluster = LocalCluster()
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://127.0.0.1:46733 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 28.00 GiB


In [19]:
history = myfuncs.get_new_log()
print(history)

Tue Jun 22 17:03:19 2021: /g/data/e14/dbi599/miniconda3/envs/unseen/bin/jupyter notebook /home/599/dbi599/unseen/notebooks/preprocess.ipynb


# CAFE

In [6]:
cafe_files = ['/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19911101/ZARR/atmos_isobaric_daily.zarr.zip',
              '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-19921101/ZARR/atmos_isobaric_daily.zarr.zip']

In [7]:
cafe_kwargs = {'metadata_file': '../config/cafe.yml',
               'no_leap_days': True,
               'region': 'TAS-POINT',
               'variables': ['pr'],
               'units' : {'pr': 'mm/day'},
              }

In [8]:
cafe_outfile = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_19911101-19921101_3650D_cafe-grid-TAS-POINT.zarr.zip'

In [10]:
cafe_ds = myfuncs.open_mfforecast(cafe_files, **cafe_kwargs)
cafe_ds.attrs['history'] = history
cafe_ds

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 2, lead_time: 3650)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
  * init_date  (init_date) datetime64[ns] 1991-11-01 1992-11-01
    time       (lead_time, init_date) datetime64[ns] 1991-11-01 ... 2002-10-29
Data variables:
    pr         (init_date, lead_time, ensemble) float64 dask.array<chunksize=(1, 28, 96), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    title:      AccessOcean-AM2
    history:    Tue Jun 22 17:00:09 2021: /g/data/e14/dbi599/miniconda3/envs/...

In [11]:
cafe_ds['pr']

<xarray.DataArray 'pr' (init_date: 2, lead_time: 3650, ensemble: 96)>
dask.array<concatenate, shape=(2, 3650, 96), dtype=float64, chunksize=(1, 28, 96), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
  * init_date  (init_date) datetime64[ns] 1991-11-01 1992-11-01
    time       (lead_time, init_date) datetime64[ns] 1991-11-01 ... 2002-10-29
Attributes:
    cell_methods:   time: mean
    interp_method:  conserve_order1
    long_name:      Total precipitation rate
    time_avg_info:  average_T1,average_T2,average_DT
    units:          kg/m2/s

In [12]:
cafe_ds = cafe_ds.chunk({'lead_time': 50})

In [13]:
cafe_ds['pr']

<xarray.DataArray 'pr' (init_date: 2, lead_time: 3650, ensemble: 96)>
dask.array<rechunk-merge, shape=(2, 3650, 96), dtype=float64, chunksize=(1, 50, 96), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
  * init_date  (init_date) datetime64[ns] 1991-11-01 1992-11-01
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(50, 2), meta=np.ndarray>
Attributes:
    cell_methods:   time: mean
    interp_method:  conserve_order1
    long_name:      Total precipitation rate
    time_avg_info:  average_T1,average_T2,average_DT
    units:          kg/m2/s

In [15]:
myfuncs.to_zarr(cafe_ds, cafe_outfile)

# AWAP

In [16]:
awap_infile = '/g/data/xv83/ds0092/data/csiro-dcfp-csiro-awap/rain_day_19000101-20201202_cafe-grid.zarr/'
awap_outfile = '/g/data/xv83/dbi599/pr_awap_cafe-grid-TAS-POINT.zarr.zip'

In [17]:
awap_kwargs = {'metadata_file': '../config/awap.yml',
               'no_leap_days': True,
               'region': 'TAS-POINT',
               'variables': ['pr'],
               'units' : {'pr': 'mm/day'},
              }

In [18]:
awap_ds = myfuncs.open_file(awap_infile, **awap_kwargs)
awap_ds.attrs['history'] = history
awap_ds

<xarray.Dataset>
Dimensions:  (time: 44136)
Coordinates:
  * time     (time) object 1900-01-01 09:00:00 ... 2020-12-02 09:00:00
Data variables:
    pr       (time) float64 dask.array<chunksize=(44136,), meta=np.ndarray>
Attributes:
    history:  Tue Jun 22 17:00:09 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [20]:
awap_ds['pr']

<xarray.DataArray 'pr' (time: 44136)>
dask.array<getitem, shape=(44136,), dtype=float64, chunksize=(44136,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1900-01-01 09:00:00 ... 2020-12-02 09:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm
    valid_range:                   [-1.0, 100000.0]

In [21]:
myfuncs.to_zarr(awap_ds, awap_outfile)